## Import libraries

In [214]:
import pandas as pd
from google.cloud import bigquery
import pickle
import os

In [197]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [215]:
client = bigquery.Client()
merch_data_ref = client.dataset('merch_store', project = 'dummy24571')

In [216]:
def bq2df(sql):
    query = client.query(sql) # API request
    results = query.result()
    return results.to_dataframe()

### Raw data

In [217]:
q = ('SELECT * FROM `dummy24571.merch_store.ga_sessions_*` '
    'LIMIT 100')

In [218]:
complete_data = bq2df(q)

In [232]:
complete_data.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,NaN,2,1496626870,1496626870,20170604,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 18, 'minu...",8822011492654883751,None,Organic Search,Not Socially Engaged
1,NaN,1,1496615015,1496615015,20170604,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",1993812746237919027,None,Organic Search,Not Socially Engaged
2,NaN,2,1496572624,1496572624,20170604,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",4011793760141066616,None,Organic Search,Not Socially Engaged
3,NaN,1,1496585762,1496585762,20170604,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",0104178955238742883,None,Organic Search,Not Socially Engaged
4,NaN,1,1496566167,1496566167,20170604,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Eastern...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 1, 'minut...",891671109788942422,None,Organic Search,Not Socially Engaged


In [78]:
merch_data = client.get_dataset(merch_data_ref)

In [147]:
tables = [x.table_id for x in client.list_tables(merch_data)]

In [148]:
len(tables)

367

In [149]:
merch_full = client.get_table(merch_data.table('ga_sessions_20160801'))

In [188]:
q = """
    SELECT 
      CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
      hitNumber,
      time,
      page.pageTitle,
      type,
      productSKU,
      v2ProductName,
      v2ProductCategory,
      productPrice/1000000 as productPrice_USD

    FROM 
      `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
      UNNEST(hits) AS hits,
      UNNEST(hits.product) AS hits_product

"""

In [189]:
dd = bq2df(q)

In [190]:
dd.head()

,visitorId,hitNumber,time,pageTitle,type,productSKU,v2ProductName,v2ProductCategory,productPrice_USD
0,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056299,Gift Card - $25.00,Home/Gift Cards/,25.00
1,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056499,Gift Card - $50.00,Home/Gift Cards/,50.00
2,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056199,Gift Card- $100.00,Home/Gift Cards/,100.00
3,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056399,Gift Card - $250.00,Home/Gift Cards/,250.00
4,8159312408158297118-151,2,289206,Men's-T-Shirts,PAGE,GGOEGAAX0104,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16.99


In [191]:
pre_process = """    
    CREATE OR REPLACE TABLE merch_store.aggregate_web_stats AS (
      WITH
        durations AS (
          --calculate pageview durations
          SELECT
            CONCAT(fullVisitorID,'-', 
                 CAST(visitNumber AS STRING),'-', 
                 CAST(hitNumber AS STRING) ) AS visitorId_session_hit,
            LEAD(time, 1) OVER (
              PARTITION BY CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING))
              ORDER BY
              time ASC ) - time AS pageview_duration
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*`,
            UNNEST(hits) AS hit 
        ),

        prodview_durations AS (
          --filter for product detail pages only
          SELECT
            CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
            productSKU AS itemId,
            IFNULL(dur.pageview_duration,
              1) AS pageview_duration,
          FROM
            `bigquery-public-data.google_analytics_sample.ga_sessions_2016*` t,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS hits_product
          JOIN
            durations dur
          ON
            CONCAT(fullVisitorID,'-',
                   CAST(visitNumber AS STRING),'-',
                   CAST(hitNumber AS STRING)) = dur.visitorId_session_hit
          WHERE
          #action_type: Product detail views = 2
          eCommerceAction.action_type = "2" 
        ),

        aggregate_web_stats AS(
          --sum pageview durations by visitorId, itemId
          SELECT
            visitorId,
            itemId,
            SUM(pageview_duration) AS session_duration
          FROM
            prodview_durations
          GROUP BY
            visitorId,
            itemId )
        SELECT
          *
        FROM
          aggregate_web_stats
    );
    -- Show table
    SELECT
      *
    FROM
      merch_store.aggregate_web_stats

"""
        

In [192]:
processed_data = bq2df(qt1)

In [193]:
processed_data.head()

,visitorId,itemId,session_duration
0,6261323644020670529-2,GGOEGAAX0031,2050
1,6633864305737833732-3,GGOEGAAX0031,2445
2,5645201504155081337-1,GGOEGAAX0031,9699
3,6501703224773598110-1,GGOEGAAX0031,88893
4,5889053612258676747-1,GGOEGAAX0031,15051


In [194]:
processed_data.agg(['count', 'size', 'nunique'])

,visitorId,itemId,session_duration
count,137514,137514,137514
size,137514,137514,137514
nunique,58531,374,38103


In [245]:
(processed_data['itemId'].nunique()).count()

AttributeError: 'int' object has no attribute 'count'

In [235]:
most_popular = popular_products.sort_values('session_duration', ascending=False)
most_popular.head(10)

,session_duration
itemId,
GGOEGAAX0104,3494
GGOEGBRJ037299,3427
GGOEGBRA037499,3056
GGOEGAAX0358,2740
GGOEGFKA022299,2616
GGOEGBRJ037399,2531
GGOEGFKQ020399,2395
GGOEGAAX0105,2176
GGOEGAAX0037,2109


## Train the matrix factorization model

In [230]:
%%bigquery

CREATE OR REPLACE MODEL merch_store.retail_recommender
OPTIONS(model_type='matrix_factorization', 
        user_col='visitorId', 
        item_col='itemId',
        rating_col='session_duration',
        feedback_type='implicit'
        )
AS
SELECT * FROM merch_store.aggregate_web_stats

Executing query with job ID: e97f143e-0908-414c-a1fd-77382a55abbb
Query executing: 0.87s


ERROR:
 400 Training Matrix Factorization models is not available for on-demand usage. To train, please set up a reservation (flex or regular) based on instructions in BigQuery public docs.

(job ID: e97f143e-0908-414c-a1fd-77382a55abbb)

              -----Query Job SQL Follows-----              

    |    .    |    .    |    .    |    .    |    .    |
   1:CREATE OR REPLACE MODEL merch_store.retail_recommender
   2:OPTIONS(model_type='matrix_factorization', 
   3:        user_col='visitorId', 
   4:        item_col='itemId',
   5:        rating_col='session_duration',
   6:        feedback_type='implicit'
   7:        )
   8:AS
   9:SELECT * FROM merch_store.aggregate_web_stats
    |    .    |    .    |    .    |    .    |    .    |
